In [20]:
import pandas as pd

df = pd.read_csv("../data/sujin-5_base_noise_detected.csv")
len(df)

2800

# 전처리

## 1. 의미 없는 문자 제거

In [ ]:
# text 앞뒤에 그래픽/게시판/종합 등의 의미 없는 단어가 나오는 경우 삭제
df["text"] = df["text"].replace(r"^(그래픽|게시판)", "", regex=True)
df["text"] = df["text"].replace(r"^(종합|1보|2보|3보)", "", regex=True)
df["text"] = df["text"].replace(r"(종합|종합1보|종합2보|1보|2보|3보)$", "", regex=True)
df.head()

,ID,text,target,is_noise
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U,4,1
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,1
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야",2,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,1


In [22]:
# 말줄임표: 띄어쓰기 처리 혹은 제거
df["text"] = df["text"].str.replace("…", " ")
df["text"] = df["text"].str.replace("...", "")
df.head()

,ID,text,target,is_noise
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U,4,1
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,1
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야",2,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,5,0
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,1


## 2. 노이즈 문장에서 특수문자 제거

In [23]:
import re
# 영어, 숫자, 한글, 한자 제외 제거
def remove_special_characters(text):
    return re.sub(r"[^a-zA-Z0-9\sㄱ-ㅎㅏ-ㅣ가-힣\u4e00-\u9fff]", "", text)

In [24]:
# 노이즈 데이터에만 함수 적용
df.loc[df["is_noise"] == 1, "text"] = df.loc[df["is_noise"] == 1, "text"].apply(
    remove_special_characters
)

df.head()

,ID,text,target,is_noise
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,5,0
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발 비해 감시 강화,6,1


## 3. 한자는 한글 뜻을 함께 기입하여 사용

In [25]:
from collections import Counter


# 한자 추출 함수 정의
def extract_hanza(text):
    # CJK 통합 한자 범위(U+4E00 ~ U+9FFF)에 해당하는 문자 추출
    return "".join(re.findall("[\u4e00-\u9fff]", text))


# 문자 빈도 축정
def get_char_frequency(text_list):
    # 모든 문자를 하나의 리스트로 합치기
    chars = [char for text in text_list for char in text]
    # Counter를 사용하여 빈도 계산
    char_counts = Counter(chars)
    # 빈도순으로 정렬
    return pd.Series(char_counts).sort_values(ascending=False)

# 한자 빈도수 측정
hanza = df["text"].apply(extract_hanza)

char_frequency = get_char_frequency(hanza)
hanza = list(char_frequency.keys())
print(len(hanza), hanza)

49 ['美', '北', '中', '朴', '靑', '日', '與', '文', '英', '野', '佛', '伊', '獨', '反', '前', '軍', '硏', '對', '外', '社', '黃', '亞', '韓', '株', '車', '崔', '院', '金', '丁', '小', '和', '企', '安', '展', '檢', '親', '銀', '證', '先', '父', '南', '詩', '家', '大', '印', '阿', '故', '州', '重']


In [26]:
hanza_hangul_meaning = [
    ('美', '미국', '나라'), ('北', '북한', '나라'), ('中', '중국', '나라'),
    ('朴', '박', '성씨'), ('靑', '청와대', '정치'), ('日', '일본', '나라'),
    ('與', '여당', '정치'), ('文', '문','성씨'), ('英', '영국', '나라'),
    ('野', '야당', '정치'), ('佛', '프랑스', '나라'), ('伊', '이탈리아', '나라'),
    ('獨', '독일', '나라'), ('反', '반', '일반'), ('前', '전', '일반'),
    ('軍', '군대', '일반'), ('硏', '연구원', '일반'), ('對', '대', '일반'),
    ('外', '외', '일반'), ('社', '회사', '경제'), ('黃', '황','성씨'),
    ('亞', '아시아', '나라'), ('韓', '한국', '나라'), ('株', '주식', '경제'),
    ('車', '차', '일반'), ('崔', '최','성씨'), ('院', '위원', '일반'),
    ('金', '김','성씨'), ('丁', '정','성씨'), ('小', '소', '일반'),
    ('和', '네덜란드', '나라'), ('企', '기업', '경제'), ('安', '안','성씨'),
    ('展', '전시', '일반'), ('檢', '검찰', '정치'), ('親', '친', '일반'),
    ('銀', '은행', '경제'), ('證', '증권', '경제'), ('先', '선', '일반'),
    ('父', '아버지', '일반'), ('南', '남한', '나라'), ('詩', '시', '일반'),
    ('家', '집안', '일반'), ('大', '대', '일반'), ('印', '인도', '나라'),
    ('阿', '아프리카', '나라'), ('故', '고', '일반'), ('州', '주', '나라'),
    ('重', '중공업', '경제')
]

def annotate_hanzi_with_meaning(text):
    for hanzi, hangul, _ in hanza_hangul_meaning:
        text = text.replace(hanzi, f"{hanzi}({hangul})")
    return text

In [27]:
df.loc[:, "text"] = df["text"].apply(annotate_hanzi_with_meaning)
df.head(10)

,ID,text,target,is_noise
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,5,0
4,ynat-v1_train_00004,pI美(미국)대선I앞두고 R2fr단 발 비해 감시 강화,6,1
5,ynat-v1_train_00005,美(미국)성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,0
6,ynat-v1_train_00006,프로야구롯TKIAs광주 경기 y천취소,1,1
7,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파 3위 굳,4,0
8,ynat-v1_train_00008,朴(박)대통령 얼마나 많이 놀라셨어요 경주 지진현장 방문,6,0
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔 알뜰폰 기대감,4,0


In [28]:
df.to_csv("3_d_2800_hanzi_dictionary.csv", index=False)

## 4. 형태소 분석기로 노이즈 제거

In [30]:
df = pd.read_csv("../data/3_d_2800_hanzi_dictionary.csv")

In [31]:
df = df[df["is_noise"]==1]
print(len(df))
df.head()

1602


,ID,text,target,is_noise
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1
4,ynat-v1_train_00004,pI美(미국)대선I앞두고 R2fr단 발 비해 감시 강화,6,1
6,ynat-v1_train_00006,프로야구롯TKIAs광주 경기 y천취소,1,1


In [32]:
# 형태소 분석기 임포트
from konlpy.tag import *
okt = Okt()

In [33]:
df["pos_tagging"] = df["text"].copy()
# 단어별 구분 없이 한 문장 통으로 하려면
# df["pos_tagging"] = df["pos_tagging"].apply(lambda text: okt.pos(text))
# 단어별로 형태소 분석 적용
df["pos_tagging"] = df["pos_tagging"].apply(str.split)
df["pos_tagging"] = df["pos_tagging"].apply(lambda li: [okt.pos(e) for e in li])

In [34]:
df.head()

,ID,text,target,is_noise,pos_tagging
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1,"[[(정, Noun), (i, Alpha)], [(파, Noun), (1, Numb..."
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1,"[[(K, Alpha), (찰국, Noun), (DLwo, Alpha)], [(로,..."
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1,"[[(m, Alpha)], [(김정, Noun)], [(자주, Noun), (통일,..."
4,ynat-v1_train_00004,pI美(미국)대선I앞두고 R2fr단 발 비해 감시 강화,6,1,"[[(pI, Alpha), (美, Foreign), ((, Punctuation),..."
6,ynat-v1_train_00006,프로야구롯TKIAs광주 경기 y천취소,1,1,"[[(프로야구, Noun), (롯, Noun), (TKIAs, Alpha), (광주..."


In [35]:
def join_words(morphs_lists):
    def filter_and_join_morphemes(morphological_results):
        # Alpha와 Punctuation을 제외한 문자만 필터링
        filtered_characters = [
            word for word, pos in morphological_results if pos not in ["Alpha", "Number"]
        ]
        # 필터링된 문자들을 공백으로 구분하여 문자열로 결합
        return "".join(filtered_characters)
    return " ".join(
        [filter_and_join_morphemes(morphs_list) for morphs_list in morphs_lists]
    )


df["pos_processed"] = df["pos_tagging"].apply(join_words)
df.head()

,ID,text,target,is_noise,pos_tagging,pos_processed
0,ynat-v1_train_00000,정i 파1 미사z KT 이용기간 2e 단 Q분종U,4,1,"[[(정, Noun), (i, Alpha)], [(파, Noun), (1, Numb...",정 파 미사 이용기간 단 분종
1,ynat-v1_train_00001,K찰국DLwo 로L3한N 회장 2 T0송,3,1,"[[(K, Alpha), (찰국, Noun), (DLwo, Alpha)], [(로,...",찰국 로한 회장 송
2,ynat-v1_train_00002,m 김정 자주통일 새r열1나가야,2,1,"[[(m, Alpha)], [(김정, Noun)], [(자주, Noun), (통일,...",김정 자주통일 새열나가야
4,ynat-v1_train_00004,pI美(미국)대선I앞두고 R2fr단 발 비해 감시 강화,6,1,"[[(pI, Alpha), (美, Foreign), ((, Punctuation),...",美(미국)대선앞두고 단 발 비해 감시 강화
6,ynat-v1_train_00006,프로야구롯TKIAs광주 경기 y천취소,1,1,"[[(프로야구, Noun), (롯, Noun), (TKIAs, Alpha), (광주...",프로야구롯광주 경기 천취소


In [36]:
df["pos_processed"] = df["pos_processed"].str.strip()
df["pos_processed"] = df["pos_processed"].apply(lambda x: re.sub(r"\s+", " ", x))

In [37]:
df = df[["ID", "pos_processed", "target","is_noise"]].rename(columns={"pos_processed": "text"})
df.to_csv("4_d_1602_only_noise.csv", index=False)

## 5. 비 노이즈 데이터 추출

In [ ]:
df = pd.read_csv("../data/3_d_2800_hanzi_dictionary.csv")
df = df[df["is_noise"]==0]
len(df)
df.to_csv("5_d_1198_only_not_noise.csv", index=False)

## 6. 비 노이즈 데이터 기반으로 BART 노이즈 복구 모델 훈련

In [59]:
import pandas as pd
df_train = pd.read_csv("../data/5_d_1198_only_not_noise.csv")

In [60]:
import random
import re

# 노이즈 문장에서 추출한 특수문자 목록
special_characters = ['…', '.', '·', '%', '"', '-', '(', '|', '?', ',', '}', ':', '&', '_', '{', '~', '#', '\\', '*', ')', '$', '=', '+', '`', ';', "'", '!', '@', '<', '/', '>', '[', ']', '^', '↑', '∼', '↓', '→', '㎜', '＋', 'ㆍ', '㎞', '㎡', 'ｍ', '②', '％', '㎝', '↔', '⅔', '㎏', '④', '③']

def introduce_noise(text: str) -> str:
    # 한글 문자 찾기
    hangul_chars = re.findall(r'[가-힣]', text)
    # 20%에 해당하는 개수 계산 (최소 1개)
    num_noise = max(1, int(len(hangul_chars) * 0.2))
    
    # 변경할 인덱스 랜덤 선택
    indexes_to_replace = random.sample(range(len(hangul_chars)), num_noise)
    
    # 텍스트 수정을 위한 리스트 변환
    noisy_text = list(text)
    
    for index in indexes_to_replace:
        # 랜덤 문자 생성 (숫자, 영어 대소문자, 정의된 특수문자)
        random_char = random.choice(
            [chr(x) for x in (
                list(range(48, 58)) +     # 숫자 (0-9)
                list(range(65, 91)) +     # 대문자 (A-Z)
                list(range(97, 123))      # 소문자 (a-z)
            )] + special_characters            # 정의된 특수문자
        )
        
        # 한글 문자를 랜덤 문자로 교체
        char_index = text.index(hangul_chars[index])
        # noisy_text[char_index] = random_char
        noisy_text[char_index] = ""
        # 사용된 한글 문자 제거 (같은 문자 중복 변경 방지)
        text = text[:char_index] + text[char_index + 1:]
    
    return ''.join(noisy_text)

In [61]:
df_train["noised_text"] = df_train["text"].copy()
df_train.loc[:,"noised_text"] = df_train["noised_text"].apply(introduce_noise)
df_train = df_train[["ID","text","noised_text"]]
df_train.head()

,ID,text,noised_text
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,갤노트8 말 27만대 개통 시장은 불법 보금 얼룩
1,ynat-v1_train_00005,美(미국)성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,美(미국)성 6 중 1명 배우자·연인 떠안은 적 있다
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파 3위 굳,가메즈 33득점 우리드 KB손해보험완파 3위 굳
3,ynat-v1_train_00008,朴(박)대통령 얼마나 많이 놀라셨어요 경주 지진현장 방문,朴(대통령 얼마나 많이 놀라셨어요 경주 지진현 문
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔 알뜰폰 기대감,얼심 아폰 하반기 출설솔솔 알뜰폰 기대감


In [67]:
import pandas as pd
df_test = pd.read_csv("../data/4_d_1602_only_noise.csv")

In [68]:
test_data = list(df_test["text"])
len(test_data)

1602

In [69]:
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split


class TextReconstructor:
    def __init__(self, model_name="gogamza/kobart-base-v2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
    
    def reconstruct(self, corrupted_text, max_length=32):
        # token_type_ids를 False로 설정
        inputs = self.tokenizer(corrupted_text, 
                              return_tensors="pt", 
                              truncation=True, 
                              max_length=max_length,
                              return_token_type_ids=False)  # 이 부분 추가
        inputs = inputs.to(self.device)
        
        outputs = self.model.generate(
            **inputs,
            max_length=max_length,
            num_beams=10,
            no_repeat_ngram_size=2,
            top_k=10,
            top_p=0.95,
            temperature=0.3
        )
        
        reconstructed = self.tokenizer.decode(outputs[0], 
                                            skip_special_tokens=True)
        return reconstructed



class TextReconstructionDataset(Dataset):
    def __init__(self, corrupted_texts, original_texts, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        print("토크나이징 데이터...")
        self.inputs = []
        self.targets = []
        
        for corrupt_text, orig_text in tqdm(zip(corrupted_texts, original_texts), 
                                          total=len(corrupted_texts),
                                          desc="데이터셋 준비중"):
            # return_token_type_ids=False 추가
            self.inputs.append(
                self.tokenizer(
                    corrupt_text,
                    truncation=True,
                    max_length=max_length,
                    padding='max_length',
                    return_tensors='pt',
                    return_token_type_ids=False
                )
            )
            self.targets.append(
                self.tokenizer(
                    orig_text,
                    truncation=True,
                    max_length=max_length,
                    padding='max_length',
                    return_tensors='pt',
                    return_token_type_ids=False
                )
            )
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"].squeeze(0),
            "attention_mask": self.inputs[idx]["attention_mask"].squeeze(0),
            "labels": self.targets[idx]["input_ids"].squeeze(0)
        }


def train_model(model, train_dataset, valid_dataset):
    training_args = TrainingArguments(
        output_dir="../outputs",
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="./logs",
        save_strategy="epoch",
        evaluation_strategy="epoch",
        logging_steps=100,
        load_best_model_at_end=True,
        report_to="none",
        # 진행 상황 표시 활성화
        disable_tqdm=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset
    )
    
    print("모델 학습 시작...")
    trainer.train()



def prepare_and_train(df):
    print("데이터 분할 중...")
    train_df, valid_df = train_test_split(
        df, 
        test_size=0.1,
        random_state=42
    )
    
    print("모델 초기화 중...")
    model_name = "gogamza/kobart-base-v2"
    reconstructor = TextReconstructor(model_name)
    
    print("학습 데이터셋 생성 중...")
    train_dataset = TextReconstructionDataset(
        corrupted_texts=train_df['noised_text'].tolist(),
        original_texts=train_df['text'].tolist(),
        tokenizer=reconstructor.tokenizer
    )
    
    print("검증 데이터셋 생성 중...")
    valid_dataset = TextReconstructionDataset(
        corrupted_texts=valid_df['noised_text'].tolist(),
        original_texts=valid_df['text'].tolist(),
        tokenizer=reconstructor.tokenizer
    )
    
    train_model(reconstructor.model, train_dataset, valid_dataset)
    
    return reconstructor

In [70]:
# 모델 학습
print("학습 프로세스 시작...")
reconstructor = prepare_and_train(df)

print("모델 저장 중...")
reconstructor.model.save_pretrained("./reconstructor/")
reconstructor.tokenizer.save_pretrained("./reconstructor")

# 테스트
print("\n=== 테스트 결과 ===")
result_data = []
for text in test_data:
    reconstructed = reconstructor.reconstruct(text)
    result_data.append(reconstructed)

학습 프로세스 시작...
데이터 분할 중...
모델 초기화 중...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


학습 데이터셋 생성 중...
토크나이징 데이터...


데이터셋 준비중: 100%|██████████| 1078/1078 [00:00<00:00, 3644.76it/s]


검증 데이터셋 생성 중...
토크나이징 데이터...


...

모델 학습 시작...


Epoch,Training Loss,Validation Loss
1,8.214600,1.705800
2,1.572300,0.211874
3,0.154700,0.191062


/opt/conda/envs/main/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


모델 저장 중...

=== 테스트 결과 ===


/opt/conda/envs/main/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/envs/main/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/conda/envs/main/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [71]:
df_test["denoised_text"] = result_data
df_test.head()

,ID,text,target,is_noise,denoised_text
0,ynat-v1_train_00000,정 파 미사 이용기간 단 분종,4,1,정정파 장거리 미사 이용기간 단기간 내 분종종기간단 추진기간도 단시종점 내달 중순 ...
1,ynat-v1_train_00001,찰국 로한 회장 송,3,1,찰스 로한 회장 송찰찰 찰 찰즈라엘 총리 송 찰찰라엘 조지아로한 회장을 송송 찰옥국...
2,ynat-v1_train_00002,김정 자주통일 새열나가야,2,1,김정은 자주통일 새 시대 열어가야 한다고 선언했다.\n
3,ynat-v1_train_00004,美(미국)대선앞두고 단 발 비해 감시 강화,6,1,美(미국)대선앞두고 단발성 비난에 비해 감시 강화키로
4,ynat-v1_train_00006,프로야구롯광주 경기 천취소,1,1,프로야구 롯데롯데롯광주 경기 천취소 승리소식삼진취소로


In [ ]:
df_test = df_test[["ID", "denoised_text", "target","is_noise"]].rename(columns={"denoised_text": "text"})
df_test["text"] = df_test["text"].str.replace("\n", "")
df_test.to_csv("6_d_1602_kobart-denoise.csv", index=False)

## 7. 데이터 결합

In [105]:
import pandas as pd
df1 = pd.read_csv("../data/5_d_1198_only_not_noise.csv")
df2 = pd.read_csv("../data/6_d_1602_kobart_denoise.csv")

df = pd.concat([df1,df2], axis=0)
df["text"] = df["text"].str.strip()
df["text"] = df["text"].apply(lambda x: re.sub(r"\s+", " ", x))

print(len(df))
df = df.sort_values(by="ID", ascending=True)
df = df[['ID', 'text', 'target']]
df.to_csv("7_d_2800_kobart_denoise.csv",index=False)

2800


## 8. cleanlab 라벨링 및 정상 라벨링 복구

In [ ]:
# 정렬
df = pd.read_csv("../data/7_dr_2800_kobart_cleanlab.csv")
df = df.sort_values(by="ID", ascending=True)
df.to_csv("../data/7_dr_2800_kobart_cleanlab.csv",index=False)

In [112]:
df1 = pd.read_csv("../data/7_dr_2800_kobart_cleanlab.csv")
df2 = pd.read_csv("../data/6_d_1602_kobart_denoise.csv")

# merge를 사용하여 ID 기준으로 데이터프레임 결합
df1 = df1.merge(df2[['ID', 'target']], on='ID', how='left', suffixes=('', '_new'))
df1['target'] = df1['target_new'].combine_first(df1['target'])
df1 = df1.drop(columns=['target_new'])

df1 = df1.sort_values(by="ID", ascending=True)
df1.to_csv("../data/7_dr_2800_kobart_cleanlab_update.csv",index=False)

## 9. 바꾸기 전 라벨과 비교

In [3]:
import pandas as pd
df_before = pd.read_csv("../data/7_d_2800_kobart_denoise.csv")
df_after = pd.read_csv("../data/7_dr_2800_kobart_cleanlab_update.csv")

In [4]:
import pandas as pd

# 두 데이터프레임을 ID와 text 컬럼을 기준으로 병합
df_merged = pd.merge(df_before, df_after, on=["ID", "text"], suffixes=("_before", "_after"))
different_targets = df_merged[df_merged["target_before"] != df_merged["target_after"]]
result_df = different_targets[['ID', 'text', 'target_before', 'target_after']]

print(len(result_df))
result_df.head()

753


,ID,text,target_before,target_after
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통 시장은 불법 보조금 얼룩,5,4.0
5,ynat-v1_train_00005,美(미국)성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,6.0
7,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파 3위 굳,4,1.0
8,ynat-v1_train_00008,朴(박)대통령 얼마나 많이 놀라셨어요 경주 지진현장 방문,6,2.0
11,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,1,5.0


In [115]:
result_df.to_csv("label_diff.csv",index=False)

## TODO 
CleanLab 첫 시도에서 라벨 오류(= 비 노이즈) 데이터는 기존 틀린 라벨 그대로 예측할 확률에 패널티를 크게 주는 방법을 생각할 수 있다.    
그럼 라벨이 틀리지 않은 200개는 어떻게 구별하는가? -> 기존 target과 새롭게 예측한 target이 같은 것 중에서 확률이 높은 순서대로 TOP 200을 뽑아서 추론할 수 있을 것이다.